**This notebook is an assignment from the course: ['Natural Language Processing with Sequence Models'](https://www.coursera.org/learn/sequence-models-in-nlp)**

**Notebook can be run [here](https://www.kaggle.com/tailender/semantic-textual-similarity-quora-question-pairs).**

In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install trax
import trax
from trax import layers as tl
from trax.supervised import training
from trax.fastmath import numpy as fastnp
import random as rnd

rnd.seed(11)

     |████████████████████████████████| 521 kB 880 kB/s eta 0:00:01
     |████████████████████████████████| 46 kB 1.7 MB/s eta 0:00:011
     |████████████████████████████████| 520 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 33.3 MB 20.3 MB/s eta 0:00:01     |████████████████████▋           | 21.5 MB 10.6 MB/s eta 0:00:02     |██████████████████████▏         | 23.1 MB 10.6 MB/s eta 0:00:01     |███████████████████████         | 24.0 MB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 214 kB 30.9 MB/s eta 0:00:01
     |████████████████████████████████| 361 kB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 1.5 MB/s  eta 0:00:01


     |████████████████████████████████| 3.4 MB 38.0 MB/s eta 0:00:01
     |████████████████████████████████| 394.7 MB 14 kB/s s eta 0:00:01    |███▌                            | 43.8 MB 35.9 MB/s eta 0:00:10     |█████                           | 62.4 MB 35.9 MB/s eta 0:00:10     |██████████                      | 123.5 MB 28.4 MB/s eta 0:00:10     |██████████▊                     | 132.7 MB 28.4 MB/s eta 0:00:10     |███████████▏                    | 138.3 MB 35.5 MB/s eta 0:00:08     |████████████████████████▉       | 305.7 MB 34.3 MB/s eta 0:00:03��████████████████████████▉   | 355.2 MB 12.5 MB/s eta 0:00:04
     |████████████████████████████████| 3.8 MB 53.9 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 57.7 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 42.1 MB/s eta 0:00:01


     |███████████████████████████████▉| 3.7 MB 52.4 MB/s eta 0:00:01

     |████████████████████████████████| 3.7 MB 52.4 MB/s eta 0:00:01
  Created wheel for jax: filename=jax-0.2.7-py3-none-any.whl size=606843 sha256=894bbb58f358f0f48615e876cf3585230c656451e677fd4a74a9dd9eefce6738
  Stored in directory: /root/.cache/pip/wheels/97/50/53/3c741874499adfb340bdb444dee5878c432673318c2df1dc86
Successfully built jax
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
  Attempting uninstall: wheel
    Found existing installation: wheel 0.34.2
    Uninstalling wheel-0.34.2:
      Successfully uninstalled wheel-0.34.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.0
    Uninstalling grpcio-1.34.0:
      Successfully uninstalled grpcio-1.34.0
  Attempting uninstall: tensorflow-estimator
    Found e

In [3]:
!unzip '../input/quora-question-pairs/train.csv.zip'

Archive:  ../input/quora-question-pairs/train.csv.zip
  inflating: train.csv               


In [4]:
df = pd.read_csv('./train.csv',low_memory=False)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
len(df)

404290

In [6]:
cut_df = int(len(df)*0.95)

df = df.sample(frac=1)

train,test = df[:cut_df],df[cut_df:]

In [8]:
len(train),len(test)

(384075, 20215)

In [9]:
train = train[train.is_duplicate==1]
len(train)

141760

In [10]:
Q1_train_words = np.array(train['question1'])
Q2_train_words = np.array(train['question2'])

Q1_test_words = np.array(test['question1'])
Q2_test_words = np.array(test['question2'])
y_test  = np.array(test['is_duplicate'])

In [41]:
vocab_dir='gs://trax-ml/vocabs/'
vocab_file='en_32k.subword'

In [105]:
# sentence = 'It is nice to learn new things today!'
Q1_train = list(map(list,list(trax.data.tokenize(iter(Q1_train_words),vocab_file=vocab_file))))
Q2_train = list(map(list,list(trax.data.tokenize(iter(Q2_train_words),vocab_file=vocab_file))))

Q1_test = list(map(list,list(trax.data.tokenize(iter(Q1_test_words),vocab_file=vocab_file))))
Q2_test = list(map(list,list(trax.data.tokenize(iter(Q2_test_words),vocab_file=vocab_file))))

In [106]:
cut_off = int(len(Q1_train)*.8)
train_Q1, train_Q2 = Q1_train[:cut_off], Q2_train[:cut_off]
val_Q1, val_Q2 = Q1_train[cut_off: ], Q2_train[cut_off:]

In [71]:
trax.data.text_encoder.PAD_ID

0

In [109]:
def data_generator(Q1, Q2, batch_size, pad=0, shuffle=True):

    input1 = []
    input2 = []
    idx = 0
    len_q = len(Q1)
    question_indexes = [*range(len_q)]
    
    if shuffle:
        rnd.shuffle(question_indexes)
    
    while True:
        if idx >= len_q:
            idx = 0
            if shuffle:
                rnd.shuffle(question_indexes)
#         print(len_q,idx)
        q1 = Q1[question_indexes[idx]]
        q2 = Q2[question_indexes[idx]]
        
        idx += 1
        
        input1.append(q1)
        input2.append(q2)
        
        if len(input1) == batch_size:
            max_len = len(max(max(input1,key=len),max(input2,key=len),key=len))
            max_len =  2**int(np.ceil(np.log2(max_len)))
#             print(max_len)
            b1 = []
            b2 = []
            for q1, q2 in zip(input1, input2):  
#                 print(q1.shape,q2.shape)
                q1 = q1+[pad]*(max_len-len(q1))                
                q2 = q2+[pad]*(max_len-len(q2))
                
                b1.append(q1)                
                b2.append(q2)
            yield np.array(b1), np.array(b2)

            input1, input2 = [], [] 

In [110]:
batch_size = 2
res1, res2 = next(data_generator(train_Q1, train_Q2, batch_size))
print("First questions  : ",'\n', res1, '\n')
print("Second questions : ",'\n', res2)

First questions  :  
 [[ 1754   938   174   485   450   174  6986    23  1539  6419  5549   144
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [ 1928  5693   183     2  3366  4330 14592  1843     6    14   974   485
    142  1603  5801 19286     3    23     2  3807    20     2  3485 14087
    918   144     0     0     0     0     0     0]] 

Second questions :  
 [[ 1754    30     2  6419  1105   144     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [ 1928  5693   183     2  3366  4330 14592  1843     6    14   974   485
    142  1603  5801 19286     3    23     2  3807    20     2  8344  4271
  14592   144     0     0     0     0     0     0]]


In [112]:
def Siamese(vocab_size=33000, d_model=128, mode='train'):

    def normalize(x):  # normalizes the vectors to have L2 norm 1
        return x / fastnp.sqrt(fastnp.sum(x * x, axis=-1, keepdims=True))
    
    q_processor = tl.Serial(  # Processor will run on Q1 and Q2.
        tl.Embedding(vocab_size,d_model), # Embedding layer
        tl.LSTM(d_model), # LSTM layer
        tl.Mean(axis=1), # Mean over columns
        tl.Fn('Normalize', lambda x: normalize(x))  # Apply normalize function
    )  # Returns one vector of shape [batch_size, d_model].
        
    # Run on Q1 and Q2 in parallel.
    model = tl.Parallel(q_processor, q_processor)
    return model

In [113]:
model = Siamese();model

Parallel_in2_out2[
  Serial[
    Embedding_33000_128
    LSTM_128
    Mean
    Normalize
  ]
  Serial[
    Embedding_33000_128
    LSTM_128
    Mean
    Normalize
  ]
]

In [114]:
def TripletLossFn(v1, v2, margin=0.25):
    """Custom Loss function.

    Args:
        v1 (numpy.ndarray): Array with dimension (batch_size, model_dimension) associated to Q1.
        v2 (numpy.ndarray): Array with dimension (batch_size, model_dimension) associated to Q2.
        margin (float, optional): Desired margin. Defaults to 0.25.

    Returns:
        jax.interpreters.xla.DeviceArray: Triplet Loss.
    """
    
    scores = fastnp.dot(v1,v2.T) 

    batch_size = len(scores)
    # use fastnp to grab all postive `diagonal` entries in `scores`
    positive = fastnp.diagonal(scores)  # the positive ones (duplicates)

    # multiply `fastnp.eye(batch_size)` with 2.0 and subtract it out of `scores`
    negative_without_positive = scores-fastnp.eye(batch_size)*2

    # take the row by row `max` of `negative_without_positive`. 
    closest_negative = negative_without_positive.max(axis=[1])

    # subtract `fastnp.eye(batch_size)` out of 1.0 and do element-wise multiplication with `scores`
    negative_zero_on_duplicate = (1-fastnp.eye(batch_size))*scores
    
    # use `fastnp.sum` on `negative_zero_on_duplicate` for `axis=1` and divide it by `(batch_size - 1)` 
    mean_negative = fastnp.sum(negative_zero_on_duplicate,axis=1)/(batch_size-1)
    # compute `fastnp.maximum` among 0.0 and `A`
    # A = subtract `positive` from `margin` and add `closest_negative` 
    triplet_loss1 = fastnp.maximum(margin-positive+closest_negative,0.0)
    
    # compute `fastnp.maximum` among 0.0 and `B`
    # B = subtract `positive` from `margin` and add `mean_negative`
    triplet_loss2 = fastnp.maximum(margin-positive+mean_negative,0)

    # add the two losses together and take the `fastnp.mean` of it
    triplet_loss = fastnp.mean(triplet_loss1+triplet_loss2)
#     print(triplet_loss)
    

    
    return triplet_loss

In [115]:
from functools import partial
def TripletLoss(margin=0.25):
    triplet_loss_fn = partial(TripletLossFn, margin=margin)
    return tl.Fn('TripletLoss', triplet_loss_fn)

In [129]:
batch_size = 256
train_generator = data_generator(train_Q1, train_Q2, batch_size)
val_generator = data_generator(val_Q1, val_Q2, batch_size)
# print('train_Q1.shape ', train_Q1.shape)
# print('val_Q1.shape   ', val_Q1.shape)

In [130]:
lr_schedule = trax.lr.warmup_and_rsqrt_decay(400, 0.01)

In [131]:
def train_model(Siamese, TripletLoss, lr_schedule, train_generator=train_generator, val_generator=val_generator, output_dir='model/'):
    """Training the Siamese Model

    Args:
        Siamese (function): Function that returns the Siamese model.
        TripletLoss (function): Function that defines the TripletLoss loss function.
        lr_schedule (function): Trax multifactor schedule function.
        train_generator (generator, optional): Training generator. Defaults to train_generator.
        val_generator (generator, optional): Validation generator. Defaults to val_generator.
        output_dir (str, optional): Path to save model to. Defaults to 'model/'.

    Returns:
        trax.supervised.training.Loop: Training loop for the model.
    """
    output_dir = os.path.expanduser(output_dir)

    train_task = training.TrainTask(
        labeled_data=train_generator,       
        loss_layer=TripletLoss(),         
        optimizer=trax.optimizers.Adam(0.01),
        lr_schedule=lr_schedule, 
    )

    eval_task = training.EvalTask(
        labeled_data=val_generator,      
        metrics=[TripletLoss()],          
    )
    
    training_loop = training.Loop(Siamese(),
                                  train_task,
                                  eval_tasks=eval_task,
                                  output_dir=output_dir)

    return training_loop

In [132]:
train_steps = 1000
training_loop = train_model(Siamese, TripletLoss, lr_schedule)
training_loop.run(n_steps = train_steps)


Step    200: Ran 100 train steps in 93.51 secs
Step    200: train TripletLoss |  0.49975640
Step    200: eval  TripletLoss |  0.49994171

Step    300: Ran 100 train steps in 81.10 secs
Step    300: train TripletLoss |  0.40659836
Step    300: eval  TripletLoss |  0.33188498

Step    400: Ran 100 train steps in 78.86 secs
Step    400: train TripletLoss |  0.29466662
Step    400: eval  TripletLoss |  0.27978107

Step    500: Ran 100 train steps in 78.90 secs
Step    500: train TripletLoss |  0.27011439
Step    500: eval  TripletLoss |  0.26462644

Step    600: Ran 100 train steps in 76.68 secs
Step    600: train TripletLoss |  0.26620609
Step    600: eval  TripletLoss |  0.25675929

Step    700: Ran 100 train steps in 75.22 secs
Step    700: train TripletLoss |  0.26236638
Step    700: eval  TripletLoss |  0.25566870

Step    800: Ran 100 train steps in 75.68 secs
Step    800: train TripletLoss |  0.25574073
Step    800: eval  TripletLoss |  0.24323204

Step    900: Ran 100 train steps 

In [124]:
def classify(test_Q1, test_Q2, y, threshold, model, data_generator=data_generator, batch_size=64):
    """Function to test the accuracy of the model.

    Args:
        test_Q1 (numpy.ndarray): Array of Q1 questions.
        test_Q2 (numpy.ndarray): Array of Q2 questions.
        y (numpy.ndarray): Array of actual target.
        threshold (float): Desired threshold.
        model (trax.layers.combinators.Parallel): The Siamese model.
        vocab (collections.defaultdict): The vocabulary used.
        data_generator (function): Data generator function. Defaults to data_generator.
        batch_size (int, optional): Size of the batches. Defaults to 64.

    Returns:
        float: Accuracy of the model.
    """
    accuracy = 0
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    for i in range(0, len(test_Q1), batch_size):
        # Call the data generator (built in Ex 01) with shuffle=False using next()
        # use batch size chuncks of questions as Q1 & Q2 arguments of the data generator. e.g x[i:i + batch_size]
        # Hint: use `vocab['<PAD>']` for the `pad` argument of the data generator
        q1, q2 = next(data_generator(test_Q1[i:i+batch_size],test_Q2[i:i+batch_size],
                                     batch_size,shuffle=False))
        # use batch size chuncks of actual output targets (same syntax as example above)
        
        y_test = y[i:i+batch_size]
#         print(y_test.shape)
        # Call the model
        v1, v2 = model([q1,q2])
#         print(v1,v2)
#         print(i)
        for j in range(batch_size):
            # take dot product to compute cos similarity of each pair of entries, v1[j], v2[j]
            # don't forget to transpose the second argument
            d = fastnp.dot(v1[j],v2[j].T)
            # is d greater than the threshold?
            res = d>threshold
#             print(j,res)
            # increment accurancy if y_test is equal `res`
            accuracy += float(y_test[j]==res)
    # compute accuracy using accuracy and total length of test questions
    accuracy = accuracy/len(test_Q1)
    ### END CODE HERE ###
    
    return accuracy

In [125]:
Q1_test = Q1_test[:512*39]
Q2_test = Q2_test[:512*39]
y_test = y_test[:512*39].tolist()

In [133]:
model = Siamese()
model.init_from_file('./model/model.pkl.gz')

In [134]:
accuracy = classify(Q1_test,Q2_test, y_test, 0.7, model, batch_size = 512)
print("Accuracy", accuracy)

Accuracy 0.717598157051282
